[Reference](https://abdulmoiz121.medium.com/twitter-sentiment-analysis-using-python-6c2f8cdbff63)

In [1]:
!pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tweepy
from tweepy import Stream
from tweepy import StreamListener
import json
from textblob import TextBlob
import re
import csv
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
consumer_key = "*********"
consumer_secret = "*********"
access_token = "************"
access_token_secret = "**********"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

posts = api.user_timeline(screen_name = "JoeBiden", count = 100, lang="en-US", tweet_mode = 'extended')
print("show 5 tweets: \n")
i = 1
for tweet in posts[0:10]:
   print(str(i) + ') ' + tweet.full_text + '\n')
   i = i+1

df = pd.DataFrame([tweet.full_text for tweet in posts], columns = ['Tweets'])
df.head()

def cleanText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#', '', text) 
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/\S+', '', text)
  
  return text
df['Tweets'] = df['Tweets'].apply(cleanText)
df

def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity
df ['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)
df
allwords = ' '.join([twts for twts in df['Tweets']])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allwords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'
df['Analysis'] = df['Polarity'].apply(getAnalysis)
df
j=1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Positive'):
     print(str(j)+ ') ' + sortedDF['Tweets'][i])
     print()
     j = j+1
j=1
sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Positive'):
     print(str(j)+ ') ' + sortedDF['Tweets'][i])
     print()
     j = j+1

ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']
round((ptweets.shape[0]) / df.shape[0] * 100, 1)

ptweets = df[df.Analysis == 'Negative']
ptweets = ptweets['Tweets']
round((ptweets.shape[0]) / df.shape[0] * 100, 1)

df['Analysis'].value_counts()
plt.title("Joe Biden Sentiment Analysis")
plt.xlabel("Sentiments")
plt.ylabel("Counts")
df['Analysis'].value_counts().plot(kind='bar')
plt.show()